# Group 10: The Potential of Professional Tennis Players: A Predictive Model for Player's Best Rank.

### Background information:
    The ATP (Association of Tennis Professionals) Tour is a worldwide tennis tour for men’s professional tennis organized annually by the ATP since 1990. The tour is divided into several tiers of tournaments, with higher tiers offering more ranking points. The PIF ATP Rankings is a merit-based method used by the ATP to rank players, determine entry qualification, and organize tournament seeding. Points are awarded based on the stage of the tournament reached and the tier. The rankings are updated weekly and rewarded points are dropped 52 weeks after being awarded, with some exceptions.


### Our research question:
    We plan to predict the best ranking of players based on various predictors. What is a player’s predicted best rank?

### The dataset:
    We will be using a data set containing player stats for the top 500 ATP players from 2017-2019. The dataset contains 37 variables, including both quantitative and qualitative values. 




## Preliminary Exploratory Data Analysis

In [1]:
# Retrieving nessesary packages
library(dplyr)
library(tidyverse)
library(rvest)
library(tidymodels)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ forcats   1.0.0     ✔ readr     2.1.4
✔ ggplot2   3.4.4     ✔ stringr   1.5.0
✔ lubridate 1.9.3     ✔ tibble    3.2.1
✔ purrr     1.0.2     ✔ tidyr     1.3.0
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors

Attaching package: ‘rvest’


The following object is masked from ‘package:readr’:

    guess_encoding


── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.

In [36]:
# Reading the data file
url = "https://drive.google.com/uc?export=download&id=1_MECmUXZuuILYeEOfonSGqodW6qVdhsS"
player_data = read_csv(url)

# Cleaning the data and getting what we need: Best Rank, 
processed_player_data = player_data |>
                        separate(Age, c("Age", "Birth date"), sep = " ") |>
                        mutate(Age = as.numeric(Age)) |>
                        separate(`Current Rank`, c("Current Rank", "Current Rank (Other)"), sep = " ") |>
                        mutate(`Current Rank` = as.numeric(`Current Rank`)) |>

                        separate(`Best Rank`, c("Best Rank", "Best Rank (Date)"), sep = " ") |>
                        mutate(`Best Rank` = as.numeric(`Best Rank`)) |>

                        separate(`Height`, c("Height (cm)", "cm REMOVE"), sep = " ") |>
                        mutate(`Height (cm)` = as.numeric(`Height (cm)`)) |>
                        separate(`Weight`, c("Weight (kg)", "kg REMOVE"), sep = " ") |>
                        mutate(`Weight (kg)` = as.numeric(`Weight (kg)`)) |>
                        mutate(Country = as.factor(Country)) |>
                        mutate(Backhand = as.factor(Backhand)) |>
                        mutate(Plays = as.factor(Plays)) |>
                        select(Age, Country, Plays, `Current Rank`, `Best Rank`, Backhand, `Height (cm)`, `Turned Pro`, Seasons, Titles, `Weight (kg)`)

# Splitting the data
player_split = initial_split(processed_player_data, prop = 3/4, strata = `Best Rank`)
player_training = training(player_split) 
player_testing = testing(player_split)

player_training
#changed

New names:
• `` -> `...1`
Rows: 500 Columns: 38
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (25): Age, Country, Plays, Wikipedia, Current Rank, Best Rank, Name, Bac...
dbl (13): ...1, Turned Pro, Seasons, Titles, Best Season, Retired, Masters, ...

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


Age,Country,Plays,Current Rank,Best Rank,Backhand,Height (cm),Turned Pro,Seasons,Titles,Weight (kg)
<dbl>,<fct>,<fct>,<dbl>,<dbl>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
32,Slovakia,Right-handed,178,44,Two-handed,185,2005,14,NA,NA
31,Argentina,Right-handed,121,3,Two-handed,198,2005,14,22,97
20,Serbia,Right-handed,54,47,Two-handed,NA,2017,3,NA,NA
22,United States,Right-handed,34,25,Two-handed,NA,2015,6,1,NA
33,Spain,Right-handed,130,19,Two-handed,190,2003,14,4,NA
32,Italy,Right-handed,12,9,Two-handed,178,2004,16,9,NA
29,Serbia,Right-handed,27,23,One-handed,180,2007,11,1,NA
29,United Kingdom,Right-handed,32,32,One-handed,175,2006,13,NA,NA
21,France,Left-handed,43,43,Two-handed,NA,2016,3,1,NA


In [ ]:
# Summary for data
data_summary = data

data_summary

In [ ]:
# Visualization for data
options(repr.plot.width=20, repr.plot.height=20)

data_plot = data |>
            ggplot(aes(x=..,y=..)) +
            geom_point() +
            xlab("x") +
            ylab("y") +
            ggTitle("x vs y") +
            theme(text = element_text(size=20))

data_plot

## Methods

### Relevant variables:
Predicted Variable: Best Rank
  
Predictors:
- Age
- Current Rank
- Height
- Year Turned pro
- Current ELO Rank
- Best ELO Rank
- Peak ELO Rating
- Weight
- Plays (Left-handed or Right-handed)
- Backhand (Two-Handed or One-Handed)
- Seasons Played
- Country


### Outlined procedure:
1. one
2. two
3. three
4. four
5. five

### Visualization:
We plan to graph a scatter plot + line plot to visualize our regression model. It will show how our 


## Expected Outcomes and Significance

### Expected findings:
We expect that a higher current rank, elo ranks, and number of seasons will generally correlate with a higher best rank for that player.

### Relevancy of findings:
Our classification model could be used to predict how future ATP tennis players will rank. These findings could also reveal correlations between the predictor variables and the predicted variable, allowing tennis players to consider how these variables may have an influence on their performance. 

### Future studies:
More specific studies, that compare predictor variables and player rank in more depth, could provide more insight into possible correlations between the variables and player performance. Additionally, we focused on top players in the ATP, but future studies could be done on a wider population of tennis players, or on more casual players.